In [1]:
import os
import urllib
import pandas as pd
import numpy as np

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/pcm-dpc/COVID-19"
COVID_PATH = os.path.join("datasets", "covid")
COVID__REGIONI_URL = DOWNLOAD_ROOT + "/master/dati-regioni/dpc-covid19-ita-regioni.csv"

def fetch_dati_regione(regioni_url=COVID__REGIONI_URL, regioni_path=COVID_PATH):
    os.makedirs(regioni_path, exist_ok=True)
    path=os.path.join(regioni_path, "regioni.csv")
    urllib.request.urlretrieve(regioni_url, path)
    
def load_dati_regione(regioni_path=COVID_PATH):
    csv_path=os.path.join(regioni_path, "regioni.csv")
    return pd.read_csv(csv_path)

In [2]:
fetch_dati_regione()
regioni=load_dati_regione()
regioni.head()



,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_attualmente_positivi,nuovi_attualmente_positivi,dimessi_guariti,deceduti,totale_casi,tamponi
0,2020-02-24 18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,0,0,0,0,0,5
1,2020-02-24 18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,0,0,0,0,0,0
2,2020-02-24 18:00:00,ITA,4,P.A. Bolzano,46.499335,11.356624,0,0,0,0,0,0,0,0,0,1
3,2020-02-24 18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,0,0,0,0,0,1
4,2020-02-24 18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,0,0,0,0,0,10


In [3]:
regioni.shape

(567, 16)

In [118]:
regioni.describe()

,codice_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_attualmente_positivi,nuovi_attualmente_positivi,dimessi_guariti,deceduti,totale_casi,tamponi
count,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000,567.000000
mean,10.190476,43.046293,12.225955,265.537919,45.271605,310.811287,245.148148,555.957672,75.275132,67.176367,50.557319,673.691358,3545.225750
std,5.800181,2.490978,2.661296,920.854589,128.884081,1047.949703,718.206548,1725.832331,200.359426,404.755919,248.150886,2348.669403,8412.625231
min,1.000000,38.115697,7.320149,0.000000,0.000000,0.000000,0.000000,0.000000,-63.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,41.125596,11.121231,2.000000,0.000000,2.000000,2.000000,5.000000,0.500000,0.000000,0.000000,5.000000,112.000000
50%,10.000000,43.616760,12.388247,17.000000,4.000000,19.000000,28.000000,50.000000,11.000000,0.000000,1.000000,52.000000,576.000000
75%,15.000000,45.434905,13.768136,113.000000,29.000000,142.500000,174.000000,348.000000,59.000000,5.000000,9.000000,370.000000,2703.500000
max,20.000000,46.499335,16.867367,8258.000000,1093.000000,9351.000000,8019.000000,17370.000000,1950.000000,5050.000000,3095.000000,25515.000000,66730.000000


In [161]:
day=regioni.data.copy()
import numpy as np
day=np.array(day)
for i in range(len(day)):
    day[i]=day[i][5:10]
day=pd.Series(day)
regioni.insert(0,column='day', value=day)

ValueError: cannot insert day, already exists

then we drop the 'data' & 'stato' column which are now useless

In [195]:
regioni=regioni.drop(['data','stato'],axis=1)

now we try to create a dataframe/dictionary which combines values of codice_regione with values of denominazione_regione

we know there are 20 regions so it is enough to access the data frame up to the 20th row. 

In [163]:
def combine_to_dict(df,upto,f1,f2):
    d12={}
    d21={}
    for i in range(upto):
        one=df.at[i,f1]
        two=df.at[i,f2]
        d12[one]=two
        d21[two]=one
    return d12,d21
d_num_as_key,d_reg_as_key=combine_to_dict(regioni,20,f1='codice_regione',f2='denominazione_regione')

now we will shorten the dataframe so that we store in another dataframe the longitude and latitude coordinates which do not change at all

In [164]:
def combine_to_datfr(df,row,col,upto):
    ind=[row[i] for i in range(upto)]
    dt=[]
    for i in range(upto):
        d=[]
        for feat in col:
            el=df.at[i,feat]
            d.append(el)
        dt.append(d)
    ret=pd.DataFrame(data=dt,index=ind,columns=col)
    return ret

new=combine_to_datfr(regioni,regioni.denominazione_regione,['lat','long'],20)    
            

# a first analysis

as we can see, we have each day a number of measurements for 21 regions (numbered from 1 to 20)

we can simply extract the number of observations of the civil protection by dividing the shape of the dataframe by 21

In [209]:
n_obs=int(regioni.shape[0]/21)
n_reg=21


27

In [181]:
def get_regione(name,period):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-181-cab8b39edd4c>, line 2)

In [235]:
def get_regione(df,name,period):
    n_obs=int(regioni.shape[0]/period)
    index=np.arange(1,n_obs+1)

    for i in range(period):
        if df.at[i,'denominazione_regione']==name:
            start=i
            break
    d=regioni[start::period]
    d=d.drop(['denominazione_regione','lat','long','codice_regione'],axis=1)
    ind={}
    a=np.arange(1,n_obs+1)
    b=np.arange(start,regioni.shape[0],step=period)
    for i in range(len(b)):
        ind[b[i]]=a[i]
    d=d.rename(index=ind)
    
    
    return d
Lom=get_regione(regioni,'Lombardia',21)
Ven=get_regione(regioni,'Veneto',21)

In [236]:
Ven

,day,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_attualmente_positivi,nuovi_attualmente_positivi,dimessi_guariti,deceduti,totale_casi,tamponi
1,02-24,12,4,16,16,32,32,0,1,33,2200
2,02-25,12,7,19,23,42,10,0,1,43,3780
3,02-26,16,8,24,45,69,27,0,2,71,4900
4,02-27,19,8,27,82,109,40,0,2,111,6164
5,02-28,24,9,33,116,149,40,0,2,151,7414
6,02-29,24,11,35,154,189,40,0,2,191,8659
7,03-01,51,13,64,197,261,72,0,2,263,9056
8,03-02,53,14,67,204,271,10,0,2,273,9782
9,03-03,49,19,68,229,297,26,7,3,307,10176
10,03-04,76,23,99,246,345,48,9,6,360,10515
